In [15]:
import os
from os.path import join as oj
import sys, time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data_pecarn
import data_psrc
import data
from data import classification_setup, get_feat_names
import train
import matplotlib.gridspec as grd

# sns.set(style="black")
# plt.style.use('dark_background')
outcome_def = 'iai_intervention' # output
%matplotlib inline
%load_ext autoreload
%autoreload 2

/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


# combined

In [49]:
df_pecarn = data_pecarn.get_data()
feat_names, pecarn_feat_names = data.get_feat_names(df_pecarn)

df_psrc = data_psrc.get_data()
feat_names, psrc_feat_names = data.get_feat_names(df_psrc)

computing psrc preprocessing...
2188


In [52]:
feat_names = [k for k in psrc_feat_names if k in pecarn_feat_names]
common_feats = feat_names + [outcome_def, 'cv_fold']
df1 = df_pecarn[common_feats]
df2 = df_psrc[common_feats]
df = pd.concat((df1, df2), axis=0)

In [ ]:
sorted(list(df_psrc.keys()))

In [53]:
common_feats

['AbdDistention_no',
 'AbdDistention_yes',
 'AbdTenderDegree_Mild',
 'AbdTenderDegree_Moderate',
 'AbdTenderDegree_Severe',
 'AbdTenderDegree_unknown',
 'Age',
 'GCSScore',
 'LtCostalTender',
 'RecodedMOI_Bike collision/fall',
 'RecodedMOI_Fall from an elevation',
 'RecodedMOI_Motor vehicle collision',
 'RecodedMOI_Motorcycle/ATV/Scooter collision',
 'RecodedMOI_Object struck abdomen',
 'RecodedMOI_Pedestrian/bicyclist struck by moving vehicle',
 'RecodedMOI_unknown',
 'RtCostalTender',
 'iai_intervention',
 'cv_fold']

In [54]:
print([k for k in psrc_feat_names if k not in pecarn_feat_names])

['AbdDistention_unknown', 'SeatBeltSign']


In [55]:
train_idxs = data.pecarn_train_idxs
test_idxs = data.psrc_train_idxs

# predict

In [ ]:
for balancing_ratio in [1, 5, 100]: 
    for balancing in ['ros', 'smote']:
         for model_type in tqdm(['logistic', 'dt']): # 'rf', 'mlp2', 'svm', 'gb'
            for num_feats in [20, 30, 52]: #111]:
                # feats = feat_names[:num_feats]
                feats = pecarn_feat_names[:num_feats]
                out_dir = f'results/full_mar15_3'
                os.makedirs(out_dir, exist_ok=True)
                out_name = f'{model_type}_{len(feats)}_{balancing}={balancing_ratio}'
                train.train(df, feat_names=feat_names, model_type=model_type, 
                            balancing=balancing, outcome_def=outcome_def,
                            balancing_ratio=balancing_ratio,
                            out_name=f'{out_dir}/{out_name}.pkl',
                            train_idxs=train_idxs,
                            test_idxs=test_idxs
                           )

/accounts/projects/vision/.local/lib/python3.7/site-packages/imblearn/utils/_validation.py:253: UserWarning: After over-sampling, the number of samples (794900) in class 1 will be larger than the number of samples in the majority class (class #0 -> 7949)
  n_samples_majority))
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/imblearn/utils/_validation.py:253: UserWarning: After over-sampling, the number of samples (790300) in class 1 will be larger than the number of samples in the majority class (class #0 -> 7903)
  n_samples_majority))
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca